In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

import openpyxl

# create a new workbook
workbook = openpyxl.Workbook()

# select the active worksheet
worksheet = workbook.active
worksheet.append(['Name', 'Company', 'Position', 'Duration', 'Link'])

# Creating a webdriver instance
driver = webdriver.Chrome("C:/Users/Lenovo/Downloads/Compressed/chromedriver/chromedriver.exe")
# This instance will be used to log into LinkedIn

# Opening linkedIn's login page
driver.get("https://linkedin.com/uas/login")

# waiting for the page to load
time.sleep(5)

# entering username
username = driver.find_element_by_id("username")
# Enter Your Email Address
username.send_keys("azharimaul@gmail.com")

# entering password
pword = driver.find_element_by_id("password")
# Enter Your Password
pword.send_keys("fmlm33otx")	

# Clicking on the log in button
# Format (syntax) of writing XPath -->
# //tagname[@attribute='value']
driver.find_element_by_xpath("//button[@type='submit']").click()
# In case of an error, try changing the
# XPath used here.

In [35]:
df = pd.DataFrame()

In [36]:
profile_urls = [line.strip() for line in open('linksdebug.txt', 'r')]

for urls in profile_urls: 

    driver.get(urls)        # this will open the link

    time.sleep(3)

    start = time.time()
    
    # will be used in the while loop
    initialScroll = 0
    finalScroll = 1000
    
    while True:
        driver.execute_script(f"window.scrollTo({initialScroll},{finalScroll}) ")
        # this command scrolls the window starting from
        # the pixel value stored in the initialScroll
        # variable to the pixel value stored at the
        # finalScroll variable
        initialScroll = finalScroll
        finalScroll += 1000
    
        # we will stop the script for 3 seconds so that
        # the data can load
        
        # You can change it as per your needs and internet speed
    
        end = time.time()
    
        # We will scroll for 20 seconds.
        # You can change it as per your needs and internet speed
        if round(end - start) > 5:
            break

        src = driver.page_source
    
        # Now using beautiful soup
        soup = BeautifulSoup(src, 'lxml')

        # Extracting the HTML of the complete introduction box
        # that contains the name, company name, and the location
        intro = soup.find('ul', {'class': 'pvs-list'})

        name = soup.find('div', {'class': 'artdeco-entity-lockup__title'})

    name = ''.join(re.split("[\n\s]{2,}", name.text)).replace("(He/Him)",'').replace("(She/Her)",'')

    experience = intro.find_all('li', {'class': 'pvs-list__paged-list-item'})

    
    n=0
    for k in experience:
        if k.find('li', {'class': ['pvs-list__paged-list-item','artdeco-list__item','pvs-list__item--line-separated']}):
            company = k.find('span', {'class': 'visually-hidden'}).text
            n=len(k.find_all('li', {'class': ['pvs-list__paged-list-item','artdeco-list__item','pvs-list__item--line-separated']}))
            continue
        else:
            if n>0:
                if not any(char.isdigit() for char in k.find_all('span', {'class': 'visually-hidden'})[1].text):
                    duration = k.find_all('span', {'class': 'visually-hidden'})[2].text
                else:
                    duration = (k.find_all('span', {'class': 'visually-hidden'})[1].text)
                df = df.append({"Nama": name,
                                "Perusahaan": company,
                                "Posisi": k.find_all('span', {'class': 'visually-hidden'})[0].text,
                                "Durasi": duration,
                                "Link": urls
                                }, ignore_index=True)
                n-=1
            else:
                try:
                    df = df.append({"Nama": name,
                                "Posisi": k.find_all('span', {'class': 'visually-hidden'})[0].text,
                                "Perusahaan": k.find_all('span', {'class': 'visually-hidden'})[1].text,
                                "Durasi": k.find_all('span', {'class': 'visually-hidden'})[2].text,
                                "Link": urls
                                }, ignore_index=True)
                except:
                    continue


    # for a,b,c,d,e in zip(names,company_list,position_list,duration_list,links):
    #     worksheet.append([a, b, c, d, e])

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7612\236273957.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Nama": name,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7612\236273957.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Nama": name,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7612\236273957.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Nama": name,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7612\236273957.py:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({"Nama": name,
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7612\236273957.py:73: Futur

In [31]:
df

,Nama,Perusahaan,Posisi,Durasi,Link
0,Didik Satrio,PT. BANK NEGARA INDONESIA (Persero) Tbk.,Product Manager of Wholesale Solution,Apr 2023 - Present · 2 mos,https://www.linkedin.com/in/didik-satrio-88777...
1,Didik Satrio,PT. BANK NEGARA INDONESIA (Persero) Tbk.,Product Analyst at Transactional Banking Servi...,Feb 2020 - Apr 2023 · 3 yrs 3 mos,https://www.linkedin.com/in/didik-satrio-88777...
2,Didik Satrio,PT. BANK NEGARA INDONESIA (Persero) Tbk.,Officer Development Program,Jun 2019 - Feb 2020 · 9 mos,https://www.linkedin.com/in/didik-satrio-88777...
3,Didik Satrio,BBM,Finance Intern,Dec 2018 - May 2019 · 6 mos,https://www.linkedin.com/in/didik-satrio-88777...
4,Raja Hafizh Al Ihsan,PT. BANK NEGARA INDONESIA (Persero) Tbk.,Data Integrity Analyst and Master Data Management,Mar 2020 - Present · 3 yrs 3 mos,https://www.linkedin.com/in/raja-hafizh-al-ihs...
...,...,...,...,...,...
579,Muhammad Ridho Albani,PT. BANK NEGARA INDONESIA (Persero) Tbk. · Con...,Bank Teller,Apr 2023 - Present · 2 mos,https://www.linkedin.com/in/muhammad-ridho-alb...
580,Muhammad Ridho Albani,Prima Food Solutions | PT Inti Prima Rasa · Co...,Barista,Jul 2022 - Mar 2023 · 9 mos,https://www.linkedin.com/in/muhammad-ridho-alb...
581,Muhammad Ridho Albani,"PT. Sumber Alfaria Trijaya, Tbk · Contract",Pramuniaga,May 2021 - May 2022 · 1 yr 1 mo,https://www.linkedin.com/in/muhammad-ridho-alb...
582,Muhammad Ridho Albani,PT Suryaprana Nutrisindo · Contract,Kurir Operasional,Nov 2020 - Feb 2021 · 4 mos,https://www.linkedin.com/in/muhammad-ridho-alb...


In [37]:
df.to_excel("profile test.xlsx", index=False)